In [94]:
import os
from dotenv import load_dotenv
load_dotenv() 

True

In [95]:
os.getenv("LANGCHAIN_PROJECT")

'AgenticAI'

In [96]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

## This value has to be set to true always
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [97]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "o1-mini")
print(model)

client=<openai.resources.chat.completions.completions.Completions object at 0x112ccc270> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x112ccc6b0> root_client=<openai.OpenAI object at 0x110163df0> root_async_client=<openai.AsyncOpenAI object at 0x112ccc380> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [98]:
result = model.invoke("What is the capital of India?")
result

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 405, 'prompt_tokens': 14, 'total_tokens': 419, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_3da8b0b088', 'id': 'chatcmpl-BcnYl0VmfvoyA3feHMYPX1Ikyza3G', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3b974d13-60de-465c-9e42-3d94a174dfaa-0', usage_metadata={'input_tokens': 14, 'output_tokens': 405, 'total_tokens': 419, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}})

In [99]:
result.content

'The capital of India is **New Delhi**.'

In [100]:
from langchain_groq import ChatGroq

model = ChatGroq(model="qwen-qwq-32b")

In [101]:
model.invoke("Hi, I am Rhythm")

AIMessage(content='\n<think>\nOkay, the user introduced themselves as Rhythm. I should respond politely and maybe ask how I can assist them today. Let me keep it friendly and open-ended so they feel comfortable to ask for help with whatever they need. Maybe something like, "Hi Rhythm! How can I assist you today?" That should work.\n\nWait, maybe I should add a bit more to make it welcoming. Maybe mention that I\'m here to help with any questions or tasks. Let me check the guidelines again to make sure I\'m following the right tone. Yep, friendly and helpful. Alright, the response should be concise but inviting. Let me go with that.\n\n</think>\n\nHi Rhythm! Nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything specific! 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 16, 'total_tokens': 187, 'completion_time': 0.390734962, 'prompt_time': 0.002900875, 'queue_ti

### PROMPT ENGINEERING

In [102]:
from langchain_core.prompts import ChatPromptTemplate

prompt =  ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI assistant. Provide me with concise and accurate answers."),
        ("user", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI assistant. Provide me with concise and accurate answers.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [103]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1134e27b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1134e2430>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [104]:
chain = prompt | model

chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI assistant. Provide me with concise and accurate answers.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1134e27b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1134e2430>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [105]:
response = chain.invoke({"input": "Can you tell me about the history of the Eiffel Tower?"})
response.content

"The Eiffel Tower was built in 1887-1889 as the entrance arch for the 1889 World's Fair in Paris, celebrating the centennial of the French Revolution. \n\nDesigned by Gustave Eiffel and his company, it was initially met with strong opposition from artists and intellectuals who considered it an eyesore. However, it quickly became a beloved symbol of Paris and a global icon. \n\nThe tower was initially meant to be dismantled after 20 years but was saved due to its use for radio transmissions.  Today, it's one of the most visited monuments in the world. \n\n"

### Output Parser

In [106]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [107]:
response = chain.invoke({"input": "What is the capital of France?"})
print(response)

Paris. 



In [108]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = JsonOutputParser()

prompt = PromptTemplate(template = "Answer the user query \n {format_instructions} \n {input} \n", input_variables = ["query"], 
                        partial_variables = {"format_instructions": output_parser.get_format_instructions()})

chain = prompt | model | output_parser

In [109]:
response = chain.invoke({"input": "What is Bitcoin in the world of Finance?"})
print(response)

{'description': 'Bitcoin is a decentralized digital currency, without a central bank or single administrator, that can be sent from user to user on the peer-to-peer bitcoin network without the need for intermediaries. Transactions are verified by network nodes through cryptography and recorded in a public distributed ledger called a blockchain.', 'key_features': ['Decentralized:', 'Digital:', 'Peer-to-peer:', 'Cryptographically secure:', 'Transparent and immutable ledger (blockchain):', 'Limited supply (21 million bitcoins)'], 'impact_on_finance': ['Alternative to traditional currencies:', 'Potential for faster and cheaper transactions:', 'New investment asset class:', 'Innovation in financial technology'], 'risks': ['Volatility:', 'Security risks:', 'Regulatory uncertainty:', 'Lack of consumer protection']}


### Using ChatPromptTemplate

In [112]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides concise and accurate answers. Provide the answer in json format."),
        ("user", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that provides concise and accurate answers. Provide the answer in json format.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [113]:
chain = prompt | model | output_parser
response = chain.invoke({"input": "What are Derivatives in Finance?"})
print(response)

{'definition': 'Financial instruments whose value is derived from an underlying asset, such as a stock, bond, commodity, or currency.', 'characteristics': ["Value fluctuates based on the underlying asset's performance.", 'Used for hedging, speculation, or arbitrage.', 'Examples include options, futures, swaps, and forwards.']}


### XML Output Parser

In [115]:
from langchain_core.output_parsers import XMLOutputParser

output_parser = XMLOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides concise and accurate answers. Provide the answer in XML format."),
        ("user", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that provides concise and accurate answers. Provide the answer in XML format.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [116]:
chain = prompt | model | output_parser
response = chain.invoke({"input": "What are Derivatives in Finance?"})
print(response)

ImportError: defusedxml is not installed. Please install it to use the defusedxml parser.You can install it with `pip install defusedxml`See https://github.com/tiran/defusedxml for more details